# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [4]:
import pandas as pd
import plotly as py
import cufflinks as cf
from ipywidgets import interact

# Importing Plotly Express for high-level, easy-to-use interactive visualizations
import plotly.express as px

cf.go_offline()

In [5]:
data = pd.read_excel("C:/Users/fee10/OneDrive/Ironhack/Module2/labs/lab-interactive-visualization/data/Online Retail.xlsx")

data.head()


,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [11]:
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])

data["Months"] = data["InvoiceDate"].dt.strftime("%m/%Y")

april_2011_df = data[(data["InvoiceDate"].dt.month == 4) & (data["InvoiceDate"].dt.year == 2011) & (data['Country'] != "United Kingdom")]


In [13]:
quantity_revenue_by_country = april_2011_df.groupby('Country')[['Quantity', 'Revenue']].sum().reset_index()

fig = px.bar(
    quantity_revenue_by_country,
    x='Country',
    y=['Quantity', 'Revenue'],
    barmode='group',
    title='Total Quantity and Revenue by Country (April 2011)',
    labels={'value': 'Total', 'variable': 'Metric'}
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [18]:
france = data[(data["InvoiceDate"] >= "2011-01-01") & (data["InvoiceDate"] <= "2011-05-31") & (data['Country'] == "France")]

quantity_revenue_france = france[['Quantity', 'Revenue']].sum()

quantity_revenue_france = quantity_revenue_france.reset_index()
quantity_revenue_france.columns = ['Metric', 'Total']
quantity_revenue_france['Country'] = 'France'

# Plotly Bar Chart
fig = px.bar(
    quantity_revenue_france,
    x='Country',
    y='Total',
    color='Metric',
    barmode='group',
    title='Total Quantity and Revenue - France (Jan–May 2011)'
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [21]:
party_bunting = data[(data['Description'] == 'PARTY BUNTING')]
party_bunting = party_bunting.groupby('Country')[['Quantity', "Revenue"]].mean().reset_index()

# Interaktiver Scatterplot
fig = px.scatter(
    party_bunting,
    x='Quantity',
    y='Revenue',
    text='Country',
    title='Average Quantity vs Revenue for PARTY BUNTING by Country',
    labels={'Quantity': 'Average Quantity', 'Revenue': 'Average Revenue'}
)

# Optionale Anpassung für bessere Lesbarkeit
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)

fig.show()

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [30]:
distribution_quantity = data[(data['Country'] == "EIRE") | (data['Country'] == "Germany") | (data['Country'] == "France") | (data['Country'] == "Netherlands")]

fig = px.histogram(
    distribution_quantity,
    x='Quantity',
    color='Country',
    title='Distribution of Quantity per Invoice by Country',
    labels={'Quantity': 'Quantity per Invoice'},
    barmode='overlay',
    histnorm='',  # Keine Normalisierung
    nbins=50  # Setze die Anzahl der Bins (z.B. 50)
)

# Zeigen des Plots
fig.show()

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [33]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [38]:
products_df = data[
    (data['Country'].isin(country_list)) & 
    (data['Description'].isin(product_list)) 
]

revenue_country = products_df.groupby(['Country', 'Description'])['Revenue'].sum().reset_index()

fig = px.bar(
    revenue_country,
    x='Country',
    y='Revenue',
    color='Description',
    barmode='group',  # Balken nebeneinander
    title='Revenue by Country and Product Description',
    labels={'Revenue': 'Total Revenue', 'Country': 'Country', 'Description': 'Product'},
)

fig.show()



## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [48]:
uk = uk.groupby('Day')['Quantity'].sum().reset_index()

fig = px.line(
    uk, 
    x='Day', 
    y='Quantity', 
    title='Total Quantity per Day', 
    labels={'Quantity': 'Total Quantity', 'Day': 'Date'},
)

fig.show()

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [55]:
uk = data[data['Country']=='United Kingdom']
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [54]:
revenue_product = data.groupby( 'Description')['Revenue'].sum().reset_index()

# Dash App erstellen
app = dash.Dash(__name__)

# Layout der App
app.layout = html.Div([
    html.H1('Revenue by Product Description'),

    # Textfeld-Widgget zur Eingabe des Textes
    dcc.Input(id='text-input', type='text', placeholder='Enter product description to filter'),

    # Balkendiagramm (Bar Chart)
    dcc.Graph(id='bar-chart'),

])

# Callback, um das Balkendiagramm basierend auf der Texteingabe zu aktualisieren
@app.callback(
    dash.dependencies.Output('bar-chart', 'figure'),
    [dash.dependencies.Input('text-input', 'value')]
)
def update_bar_chart(filter_text):
    # Wenn der Text im Textfeld leer ist, filtere nicht
    if filter_text:
        filtered_data = revenue_product[revenue_product['Description'].str.contains(filter_text, case=False, na=False)]
    else:
        filtered_data = revenue_product

    # Erstelle das Balkendiagramm
    fig = px.bar(
        filtered_data,
        x='Description',
        y='Revenue',
        title='Revenue by Product Description',
        labels={'Description': 'Product Description', 'Revenue': 'Revenue'},
    )
    
    # Zeige das Diagramm
    return fig

# Starte die Dash-App
if __name__ == '__main__':
    app.run(debug=True)